# Dataset transformation for LWU San Jacinto Readings Dataset

In this notebook, the consolidated dataset is transformed into wide format, containing the cleaned consumption data.

In [4]:
from pathlib import Path
import os
import pandas as pd

base_dir = Path("../../dataset/clean/training/pre_production")
years = os.listdir(base_dir)

columns = ['Control Number', 'Cleaned Consumption']
all_data = []

for year in years:
    files = os.listdir(base_dir / year)
    for file in files:
        month = file.split("_")[0]
        df = pd.read_csv(base_dir / year / file, usecols=columns)
        df['Month_Year'] = f"{year}_{month}"
        all_data.append(df)

# Combine into a single DataFrame
df_all = pd.concat(all_data, ignore_index=True)

# Pivot to wide format
wide_df = df_all.pivot_table(
    index=['Control Number'],
    columns='Month_Year',
    values='Cleaned Consumption',
).reset_index()

wide_df = wide_df.sort_index(axis=1)

# Optional: Sort columns so months are in order
print(wide_df.head())


Month_Year  2019_DEC  2020_APR  2020_AUG  2020_DEC  2020_FEB  2020_JAN  \
0                NaN       NaN       NaN       NaN       NaN       NaN   
1                NaN       NaN       NaN       NaN       NaN       NaN   
2                NaN       NaN       NaN       NaN       NaN       NaN   
3                0.0       0.0       0.0       0.0       0.0       0.0   
4                0.0       0.0       0.0       0.0       0.0       0.0   

Month_Year  2020_JUL  2020_JUN  2020_MAR  2020_MAY  ...  2024_MAY  2024_NOV  \
0                NaN       NaN       NaN       NaN  ...      48.0       NaN   
1                NaN       NaN       NaN       NaN  ...       NaN       0.0   
2                NaN       NaN       NaN       NaN  ...      12.0       NaN   
3                0.0       0.0       0.0       0.0  ...       0.0       0.0   
4                0.0       0.0       0.0       0.0  ...      18.0      20.0   

Month_Year  2024_OCT  2024_SEP  2025_APR  2025_FEB  2025_JAN  2025_MAR  \
0     

In [7]:
# ================================================
# Consolidate Wide Data with Connection Status and Last Reading
# ================================================

import calendar
import os
from utilities import read_csv, save_csv

# ------------------------------------------------
# 1. Read source datasets
# ------------------------------------------------
right_df = read_csv("../../dataset/clean/training/pre_production/2025/MAY_2025_pre_production.csv")  
name_df = read_csv("../../dataset/clean/training/semi_clean/cleaned_master.csv") 

# ------------------------------------------------
# 2. Merge connection status from pre-production
# ------------------------------------------------
final_df = wide_df.merge(
    right_df[['Control Number', 'District', 'Street', 'Connection Status', 'Connection Type']],
    on='Control Number',
    how='left'
)

# Normalize connection status values
final_df['Connection Status'] = (
    final_df['Connection Status']
    .replace({"Invalid Reading": "Active"})  # Convert "Invalid Reading" to "Active"
    .fillna("Unknown")                       # Fill missing with "Unknown"
)

# ------------------------------------------------
# 3. Identify month columns (exclude non-monthly fields)
# ------------------------------------------------
fixed_cols_pre_sort = [col for col in final_df.columns if not col[0].isdigit()]  # Columns that are not YYYY_MON format
month_cols = [col for col in final_df.columns if col not in fixed_cols_pre_sort]

# ------------------------------------------------
# 4. Sort month columns chronologically
# ------------------------------------------------
month_map = {m.upper(): i for i, m in enumerate(calendar.month_abbr) if m}  # {"JAN":1, "FEB":2, ...}

month_cols_sorted = sorted(
    month_cols,
    key=lambda x: (int(x.split('_')[0]), month_map[x.split('_')[1].upper()])
)

# ------------------------------------------------
# 5. Final column ordering: fixed fields first, then months
# ------------------------------------------------
fixed_cols_final = ['Control Number', 'District', 'Street', 'Connection Type', 'Connection Status']
final_df = final_df[fixed_cols_final + month_cols_sorted]

# ------------------------------------------------
# 6. Save the consolidated output
# ------------------------------------------------
save_path = "../../dataset/clean/training/production/tabular/consumption_consolidated.csv"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
save_csv(final_df, save_path)

# ------------------------------------------------
# 7. Preview output for sanity check
# ------------------------------------------------
display(final_df.head())
display(final_df.info())


,Control Number,District,Street,Connection Type,Connection Status,2019_DEC,2020_JAN,2020_FEB,2020_MAR,2020_APR,...,2024_AUG,2024_SEP,2024_OCT,2024_NOV,2024_DEC,2025_JAN,2025_FEB,2025_MAR,2025_APR,2025_MAY
0,0.0,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,121.0,District 2,Esparrago Street,Residential,Disconnected,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,50964.0,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,500001.0,District 4,Esparrago Street,Residential,Disconnected,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,500002.0,District 4,Esparrago Street,Residential,Active,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,20.0,0.0,0.0,0.0,0.0,0.0,153.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2180 entries, 0 to 2179
Data columns (total 59 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Control Number     2180 non-null   float64
 1   District           1880 non-null   object 
 2   Street             2167 non-null   object 
 3   Connection Type    2167 non-null   object 
 4   Connection Status  2180 non-null   object 
 5   2019_DEC           1558 non-null   float64
 6   2020_JAN           1563 non-null   float64
 7   2020_FEB           1606 non-null   float64
 8   2020_MAR           1619 non-null   float64
 9   2020_APR           1619 non-null   float64
 10  2020_MAY           1621 non-null   float64
 11  2020_JUN           1619 non-null   float64
 12  2020_JUL           1621 non-null   float64
 13  2020_AUG           1627 non-null   float64
 14  2020_SEP           1694 non-null   float64
 15  2020_OCT           1695 non-null   float64
 16  2020_NOV           1695 

None